In [3]:
import mujoco
import numpy as np
from sim.integrators import LGVI
from sim.env_mj import MujocoEnv

In [4]:
model = "../models/resizable_box.xml"
mode = 'custom'
dt = 0.002
integ = LGVI(m=1.0, J=np.diag([0.01, 0.01, 0.01])) if mode == "custom" else None
env = MujocoEnv(model_path=model, dt=dt, integrator=integ,mode=mode, base_body_name="box")

In [ ]:
def alpha0(A,B,C):
    if A<0 or B<0 or C<0:
        print('Please provide positive values')
    return np.arccos(np.sqrt( A*(B-C) / (B*(A-C)) ))

def alpha(h,Ekin,A,C):
    if h<0 or Ekin<0 or A<0 or C<0:
        print('Please provide positive values')
    return np.arccos(np.sqrt( (h**2/C-2*Ekin) / (h**2/C-h**2/A) ))

def check_rot_mode(alpha0,alpha,Dpsi):
    if alpha > alpha0:
        print('System in twisting mode.')
        if Dpsi==0:
            print('Specify a positive Dpsi.')
            return False
        return True
    else:
        print('System in wobbling mode.')
        if Dpsi!=0:
            print('Dpsi needs to be 0.')
            return False
        return True

def theta_integration()

def phase_times(Dphis,Dpsis,A,B,C):
    for i in range(len(Dphis)):
        Dphi = Dphis[i]
        Dpsi = Dpsis[i]
        if Dpsi==0:
            print('System in wobbling mode.')
            #



In [ ]:
# Verify the integrals and plot the time profiles for the two solutions
import numpy as np
import matplotlib.pyplot as plt

import mpmath as mp
import numpy as np

def simulate_one_period(alpha, p, m, A, C, h=1.0, N=2000):
    """
    Simulate one nutation period for the Euler top (dn case).

    Parameters
    ----------
    alpha : float (radians)
    p     : float > 0          # time scale from the dn solution
    m     : float in (0,1)     # elliptic parameter (k^2)
    A, C  : floats             # principal moments (A != C)
    h     : float              # prefactor from your equations (often 1)
    N     : int                # number of time samples over one period

    Returns
    -------
    t, theta, phidot, psidot : numpy arrays of length N
      t       : time over one nutation period
      theta   : tilt angle satisfying sin(theta)=sin(alpha)*dn(p*t|m)
      phidot  : instantaneous rate for φ
      psidot  : instantaneous rate for ψ
    """
    D = 1.0/C - 1.0/A
    K = mp.ellipk(m)
    T = float(2*K/p)
    t = np.linspace(0.0, T, N)

    s  = float(mp.sin(alpha))
    c2 = float(mp.cos(alpha)**2)

    def dn(u):
        return float(mp.ellipfun('dn', u, m))

    theta  = np.empty_like(t)
    phidot = np.empty_like(t)
    psidot = np.empty_like(t)

    for i, tt in enumerate(t):
        u = p*tt
        dn_u = dn(u)
        x = s*dn_u
        # guard against tiny numerical overshoot before arcsin
        if x > 1.0:  x = 1.0
        if x < -1.0: x = -1.0
        theta[i] = np.arcsin(x)

        denom = 1.0 - (s*s)*(dn_u*dn_u)
        phidot[i] = (h/A) * c2 / denom
        psidot[i] = h*D * c2 / np.sqrt(denom)

    return t, theta, phidot, psidot

def check_and_plot(solutions, A, C, h):
    fig_theta, fig_phidot, fig_psidot = plt.figure(), plt.figure(), plt.figure()

    for m, alpha, p, T in solutions:
        t, theta, phidot, psidot = simulate_one_period(alpha, p, m, A, C, h, N=3000)
        # Integrals over a period
        Delta_phi_num = np.trapz(phidot, t)
        Delta_psi_num = np.trapz(psidot, t)
        print(f"m={m:.2f}:  alpha={alpha:.6f}, p={p:.6f}, T={T:.6f},  Δφ≈{Delta_phi_num:.6f},  Δψ≈{Delta_psi_num:.6f}")

        # Normalized time for overlay
        x = t/t[-1]

        plt.figure(fig_theta.number)
        plt.plot(x, theta, label=f"m={m:.2f}, T≈{T:.2f}")

        plt.figure(fig_phidot.number)
        plt.plot(x, phidot, label=f"φ̇, m={m:.2f}")

        plt.figure(fig_psidot.number)
        plt.plot(x, psidot, label=f"ψ̇, m={m:.2f}")

    plt.figure(fig_theta.number)
    plt.xlabel("normalized time within nutation period")
    plt.ylabel("θ(t) [rad]")
    plt.title("Tilt angle θ(t) over one nutation cycle")
    plt.legend()
    plt.show()

    plt.figure(fig_phidot.number)
    plt.xlabel("normalized time within nutation period")
    plt.ylabel("φ̇(t) [rad/s] (arb. units)")
    plt.title("φ̇(t) over one nutation cycle")
    plt.legend()
    plt.show()

    plt.figure(fig_psidot.number)
    plt.xlabel("normalized time within nutation period")
    plt.ylabel("ψ̇(t) [rad/s] (arb. units)")
    plt.title("ψ̇(t) over one nutation cycle")
    plt.legend()
    plt.show()

check_and_plot(solutions, A_demo, C_demo, h_demo)